# Imports

In [ ]:
import requests
import time
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Variables

In [ ]:
clientID = '846e46c84e5a4b8d97026e419da77447'
clientSecret = '47f03d8d55e6417ab8dcd727e9f68a33'
spot = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(clientID,clientSecret))

In [ ]:
name = "King Gizzard"
# gizID = "6XYvaoDGE0VmRt83Jss9Sn"
# gizURI = f"spotify:artist:{gizID}"
# gizURL = f"https://api.spotify.com/v1/artist/{gizID}"

# Functions

In [ ]:
def getArtist(spotify,name):
    results = spotify.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    id = items[0]['id']
    uri = items[0]['uri']
    url = f"https://api.spotify.com/v1/artist/{id}"
    return id,uri,url

In [ ]:
def getAlbums(spotify,URL):
    results = spotify.artist_albums(URL, country = "US", album_type='album')
    albums = results['items']
    while results['next']:
        results = spotify.next(results)
        albums.extend(results['items'])
    ids =[]
    uris = []
    names = []
    dates = []
    images = []
    tracks = []

    for album in albums:
        ids.append(album['id'])
        uris.append(album['uri'])
        names.append(album['name'])
        dates.append(album['release_date'])
        images.append(album['images'])
        tracks.append(album['total_tracks'])

    df = pd.DataFrame()
    df['AID'] = ids
    df['URI'] = uris
    df['Name'] = names
    df['Release Date'] = dates
    df['Track Count'] = tracks
    df['Album Art'] = images
    return df

In [ ]:
def getTracks(spotify,album):
    results = spotify.album_tracks(album['URI'])
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    ids = []
    uris = []
    trackNo = []
    names = []
    aids = []
    for track in tracks:
        ids.append(track['id'])
        uris.append(track['uri'])
        trackNo.append(track['track_number'])
        names.append(track['name'])
        aids.append(album['AID'])
    df = pd.DataFrame()
    df['SID'] = ids
    df['URI'] = uris
    df['Track Number'] = trackNo
    df['Name'] = names
    df['AID'] = aids
    return df

# Search For GizzID on Spotify

In [ ]:
gizID, gizURI, gizURL = getArtist(spot,name)

# Get all Gizz Albums

In [ ]:
albums = getAlbums(spot,gizURL)
albums = albums[albums['AID']!='2e5TG0TyL4Q1HvW4w2O8FP']
albums = albums[albums['AID']!='6yDDniLbk2UxkUl7cjpkWE']
albums.head(30)

In [ ]:
allTracks = pd.DataFrame()
for index,row in albums.iterrows():
    allTracks = pd.concat([allTracks,(getTracks(spot,row))],axis=0)
allTracks.head(100)

# Scratch